# Creating Model Card for Toxic Comments Classification in Tensorflow

Adapted form [Tensorflow](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/pc/exercises/fairness_text_toxicity_part1.ipynb?utm_source=practicum-fairness&utm_campaign=colab-external&utm_medium=referral&utm_content=fairnessexercise1-colab#scrollTo=2z_xzJ40j9Q-) 

#### Training Dependnecies

In [1]:
import os
import tempfile
import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv

from tensorflow_model_analysis.addons.fairness.post_export_metrics import fairness_indicators
from tensorflow_model_analysis.addons.fairness.view import widget_view

#### Model Card Dependencies

In [2]:
from model_card_gen.model_card_gen import ModelCardGen
from model_card_gen.datasets import TensorflowDataset

## Download Data

#### Data Description

This version of the CivilComments Dataset provides access to the primary seven labels that were annotated by crowd workers, the toxicity and other tags are a value between 0 and 1 indicating the fraction of annotators that assigned these attributes to the comment text.

The other tags are only available for a fraction of the input examples. They are currently ignored for the main dataset; the CivilCommentsIdentities set includes those labels, but only consists of the subset of the data with them. The other attributes that were part of the original CivilComments release are included only in the raw data. See the Kaggle documentation for more details about the available features.

The comments in this dataset come from an archive of the Civil Comments platform, a commenting plugin for independent news sites. These public comments were created from 2015 - 2017 and appeared on approximately 50 English-language news sites across the world. When Civil Comments shut down in 2017, they chose to make the public comments available in a lasting open archive to enable future research. The original data, published on figshare, includes the public comment text, some associated metadata such as article IDs, timestamps and commenter-generated "civility" labels, but does not include user ids. Jigsaw extended this dataset by adding additional labels for toxicity, identity mentions, as well as covert offensiveness. This data set is an exact replica of the data released for the Jigsaw Unintended Bias in Toxicity Classification Kaggle challenge. This dataset is released under CC0, as is the underlying comment text.

For comments that have a parent_id also in the civil comments data, the text of the previous comment is provided as the "parent_text" feature. Note that the splits were made without regard to this information, so using previous comments may leak some information. The annotators did not have access to the parent text when making the labels.

*source*: https://www.tensorflow.org/datasets/catalog/civil_comments

```
@misc{pavlopoulos2020toxicity,
    title={Toxicity Detection: Does Context Really Matter?},
    author={John Pavlopoulos and Jeffrey Sorensen and Lucas Dixon and Nithum Thain and Ion Androutsopoulos},
    year={2020}, eprint={2006.00998}, archivePrefix={arXiv}, primaryClass={cs.CL}
}

@article{DBLP:journals/corr/abs-1903-04561,
  author    = {Daniel Borkan and
               Lucas Dixon and
               Jeffrey Sorensen and
               Nithum Thain and
               Lucy Vasserman},
  title     = {Nuanced Metrics for Measuring Unintended Bias with Real Data for Text
               Classification},
  journal   = {CoRR},
  volume    = {abs/1903.04561},
  year      = {2019},
  url       = {http://arxiv.org/abs/1903.04561},
  archivePrefix = {arXiv},
  eprint    = {1903.04561},
  timestamp = {Sun, 31 Mar 2019 19:01:24 +0200},
  biburl    = {https://dblp.org/rec/bib/journals/corr/abs-1903-04561},
  bibsource = {dblp computer science bibliography, https://dblp.org}
}

@inproceedings{pavlopoulos-etal-2021-semeval,
    title = "{S}em{E}val-2021 Task 5: Toxic Spans Detection",
    author = "Pavlopoulos, John  and Sorensen, Jeffrey  and Laugier, L{'e}o and Androutsopoulos, Ion",
    booktitle = "Proceedings of the 15th International Workshop on Semantic Evaluation (SemEval-2021)",
    month = aug,
    year = "2021",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2021.semeval-1.6",
    doi = "10.18653/v1/2021.semeval-1.6",
    pages = "59--69",
}

```

**Feature documentation**:

|Feature|Class|Dtype|
|-------|:---:|:---:|
|article_id|	Tensor|		tf.int32|
|id|	Tensor|		tf.string|
|identity_attack|	Tensor|		tf.float32|
|insult|	Tensor|		tf.float32|
|obscene|	Tensor|		tf.float32|
|parent_id|	Tensor|		tf.int32|
|parent_text|	Text|		tf.string|
|severe_toxicity|	Tensor|		tf.float32|
|sexual_explicit|	Tensor|		tf.float32|
|text|	Text|		tf.string|
|threat|	Tensor|		tf.float32|
|toxicity|	Tensor|		tf.float32|

In [3]:
dataset_url = 'https://storage.googleapis.com/civil_comments_dataset/'

train_tf_file = tf.keras.utils.get_file('train_tf_processed.tfrecord',
                                        dataset_url + 'train_tf_processed.tfrecord')

validate_tf_file = tf.keras.utils.get_file('validate_tf_processed.tfrecord',
                                           dataset_url + 'validate_tf_processed.tfrecord')

### Analyze Data

In [4]:
stats = tfdv.generate_statistics_from_tfrecord(data_location=train_tf_file)
tfdv.visualize_statistics(stats)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


## Train Model

In [5]:
TEXT_FEATURE = 'comment_text'
LABEL = 'toxicity'

FEATURE_MAP = {
    LABEL: tf.io.FixedLenFeature([], tf.float32),
    TEXT_FEATURE: tf.io.FixedLenFeature([], tf.string),
    
    'sexual_orientation': tf.io.VarLenFeature(tf.string),
    'gender': tf.io.VarLenFeature(tf.string),
    'religion': tf.io.VarLenFeature(tf.string),
    'race': tf.io.VarLenFeature(tf.string),
    'disability': tf.io.VarLenFeature(tf.string)
}

In [6]:
def train_input_fn():
    def parse_function(serialized):
        # parse_single_example works on tf.train.Example type
        parsed_example = tf.io.parse_single_example(serialized=serialized, features=FEATURE_MAP)
        # fighting the 92%-8% imbalance in the dataset
        # adding `weight` label, doesn't exist already (only FEATURE_MAP keys exist)
        parsed_example['weight'] = tf.add(parsed_example[LABEL], 0.1)  # 0.1 for non-toxic, 1.1 for toxic
        return (parsed_example, parsed_example[LABEL])  # (x, y)
    

    train_dataset = tf.data.TFRecordDataset(filenames=[train_tf_file]).map(parse_function).batch(512)
    return train_dataset

#### Build Model

In [7]:
# vectorizing through TFHub
embedded_text_feature_column = hub.text_embedding_column(
    key=TEXT_FEATURE,
    module_spec='https://tfhub.dev/google/nnlm-en-dim128/1')

classifier = tf.estimator.DNNClassifier(
    hidden_units=[500, 100],
    weight_column='weight',
    feature_columns=[embedded_text_feature_column],
    optimizer=tf.optimizers.Adagrad(learning_rate=0.003),
    loss_reduction=tf.losses.Reduction.SUM,
    n_classes=2)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2iqlxs2p', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2iqlxs2p', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


#### Train Model

In [8]:
classifier.train(input_fn=train_input_fn, steps=1000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.
2022-06-16 20:29:16.278596: W tensorflow/core/common_runtime/graph_constructor.cc:1511] Importing a graph with a lower producer version 


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 1000 into /tmp/tmp2iqlxs2p/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 1000...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 1000...


INFO:tensorflow:Loss for final step: 50.80476.


INFO:tensorflow:Loss for final step: 50.80476.


## Export in EvalSavedModel Format

In [9]:
MODEL_PATH = tempfile.gettempdir()

def eval_input_receiver_fn():
    serialized_tf_example = tf.compat.v1.placeholder(dtype=tf.string, shape=[None], name='input_example_placeholder')
    
    receiver_tensors = {'examples': serialized_tf_example}
    features = tf.io.parse_example(serialized_tf_example, FEATURE_MAP)
    features['weight'] = tf.ones_like(features[LABEL])
    
    return tfma.export.EvalInputReceiver(
        features=features,
        receiver_tensors=receiver_tensors,
        labels=features[LABEL]
    )

tfma_export_dir = tfma.export.export_eval_savedmodel(
    estimator = classifier,  # trained model
    export_dir_base = MODEL_PATH,
    eval_input_receiver_fn = eval_input_receiver_fn
)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets written to: /tmp/temp-1655436623/assets


INFO:tensorflow:Assets written to: /tmp/temp-1655436623/assets


INFO:tensorflow:SavedModel written to: /tmp/temp-1655436623/saved_model.pb


INFO:tensorflow:SavedModel written to: /tmp/temp-1655436623/saved_model.pb


In [10]:
# export EvalSavedModel 
tfma_export_dir = tfma.export.export_eval_savedmodel(
    estimator = classifier,  # trained model
    export_dir_base = MODEL_PATH,
    eval_input_receiver_fn = eval_input_receiver_fn
)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2022-06-16 20:30:25.677260: W tensorflow/core/common_runtime/graph_constructor.cc:1511] Importing a graph with a lower producer version 
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets written to: /tmp/temp-1655436625/assets


INFO:tensorflow:Assets written to: /tmp/temp-1655436625/assets


INFO:tensorflow:SavedModel written to: /tmp/temp-1655436625/saved_model.pb


INFO:tensorflow:SavedModel written to: /tmp/temp-1655436625/saved_model.pb


## Making a Model Card

In [11]:
_model_path = tfma_export_dir
_data_paths = {'eval': TensorflowDataset(validate_tf_file),
               'train': TensorflowDataset(train_tf_file)}

In [12]:
_eval_config =  'eval_config.proto'

In [16]:
%%writefile {_eval_config}

model_specs {
# To use EvalSavedModel set `signature_name` to "eval".
signature_name: "eval"
}

## Post training metric information. These will be merged with any built-in
## metrics from training.
metrics_specs {
metrics { class_name: "BinaryAccuracy" }
metrics { class_name: "Precision" }
metrics { class_name: "Recall" }
metrics { class_name: "ConfusionMatrixPlot" }
metrics { class_name: "FairnessIndicators" }
}

## Slicing information
slicing_specs {}  # overall slice
slicing_specs {
feature_keys: ["gender"]
}

Writing eval_config.proto


In [14]:
mc = {
  "model_details": {
    "name": "Detecting Toxic Comments",
    "overview":  (
    'The Conversation AI team, a research initiative founded by Jigsaw and Google '
    '(both part of Alphabet), builds technology to protect voices in conversation. '
    'A main area of focus is machine learning models that can identify toxicity in '
    'online conversations, where toxicity is defined as anything *rude, disrespectful '
    'or otherwise likely to make someone leave a discussion*. '
    'This multi-headed model attemps to recognize toxicity and several subtypes of toxicity: '
    'This model recognizes toxicity and minimizes this type of unintended bias '
    'with respect to mentions of identities. Reduce unintended bias ensured we can detect toxicity '
    ' accross a wide range of conversations. '),
    "owners": [
      {
        "name": "Intel XAI Team",
        "contact": "xai@intel.com"
      }
    ],

    "references": [
      {
        "reference": "https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data"
      },
      {
        "reference": "https://medium.com/jigsaw/unintended-bias-and-names-of-frequently-targeted-groups-8e0b81f80a23"
      }
    ],
    "graphics": {
      "description": " "
    }
  },
  "considerations": { 
      "limitations": [
            {"description": ('Overrepresented Identities in Data:\n'
                    'Identity terms for more frequently targeted groups '
                   '(e.g. words like “black”, “muslim”, “feminist”, “woman”, “gay” etc)'
                   ' often have higher scores because comments about those groups are '
                   'over-represented in abusive and toxic comments.')
            },
           {"description": ('False Positive Rate:\n'
                    'The names of targeted groups appear far more often in abusive '
                    'comments. For example, in many forums unfortunately it’s common '
                    'to use the word “gay” as an insult, or for someone to attack a '
                    'commenter for being gay, but it is much rarer for the word gay to '
                    'appear in a positive, affirming statements (e.g. “I am a proud gay man”). '
                    'When the training data used to train machine learning models contain these '
                    'comments, ML models adopt the biases that exist in these underlying distributions, '
                    'picking up negative connotations as they go. When there’s insufficient diversity '
                    'in the data, the models can over-generalize and make these kinds of errors.')
            },
           {"description": ('Imbalenced Data:\n'
                     'We developed new ways to balance the training '
                     'data so that the model sees enough toxic and non-toxic examples '
                     'containing identity terms in such a way that it can more effectively '
                     'learn to distinguish toxic from non-toxic uses. You can learn more '
                     'about this in our paper published at the AI, Ethics, and Society Conference.')
            },
        ]
    },
    
  "quantitative_analysis": {
    "graphics": {
      "description": " "
    }
  },
  "schema_version": "0.0.1"
}

In [17]:
mcg = ModelCardGen.generate(_data_paths, _model_path, _eval_config, model_card=mc)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


INFO:tensorflow:Restoring parameters from /tmp/1655436625/variables/variables


INFO:tensorflow:Restoring parameters from /tmp/1655436625/variables/variables


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from_tensor_info or tf.compat.v1.saved_model.get_tensor_from_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from_tensor_info or tf.compat.v1.saved_model.get_tensor_from_tensor_info.


In [18]:
mcg